In [1]:
using JuMP
using SumOfSquares
using DynamicPolynomials
using SCS

In [ ]:

N = 10
d = 2
@polyvar x[1:N,1:d]
X = monomials(reshape(x,d*N), 0:2)
X1 = monomials(reshape(x,d*N), 0:4)

model = SOSModel(with_optimizer(SCS.Optimizer))

acc = nothing
for t in sum(x .* x, dims=2)
    #print(t)
    p = @variable(model, [1:1], Poly(X1))
    #print(p)
    if acc != nothing
        acc += p * (t - 1)
    else
        acc = p * (t - 1)
    end
end

for i in range(1,stop=N)
    for j in range(1,stop=i-1)
        d  = x[i,:] - x[j,:]
        p = @variable(model, [1:1], SOSPoly(X))
        acc += p * (sum(d .* d) - 1)
    end
end

#print(acc)
print(typeof(acc))
@constraint(model, acc[1] == -1 )
optimize!(model)

┌ Warning: The addition operator has been used on JuMP expressions a large number of times. This warning is safe to ignore but may indicate that model generation is slower than necessary. For performance reasons, you should not add expressions in a loop. Instead of x += y, use add_to_expression!(x,y) to modify x in place. If y is a single variable, you may also use add_to_expression!(x, coef, y) for x += coef*y.
└ @ JuMP /Users/philip/.julia/packages/JuMP/MsUSY/src/JuMP.jl:747


1-element Array{UnitRange{Int64},1}:
 1:1